In [51]:
import cv2
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import cv2
import numpy as np
from imutils.video import VideoStream
import random as rd

# First let set some parameter for path files

Please down load the weights file and config file from `https://github.com/kiyoshiiriemon/yolov4_darknet`

In [ ]:
path_to_weights = "./yolov4.weights"
path_to_config = "./yolov4.cfg"
path_to_classname = "./classnames.txt"

# Intrusion Warning with Objects detection
Using yolo version 4 with pretrained weights

In [52]:
class YoloDetect():
    def __init__(self, detect_class=["person"], img_size=(416, 416), zoom=1, only_detect_object=False):
        # Parameters
        self.weights_file = path_to_weights
        self.config_file = path_to_config
        self.classes_file = path_to_classname
        self.conf_threshold = 0.5
        self.nms_threshold = 0.4
        self.detect_class = detect_class
        self.scale = 1/255
        self.only_detect_object = only_detect_object
        self.img_size = img_size
        self.zoom = zoom
        self.net = cv2.dnn.readNetFromDarknet('yolov4.cfg', 'yolov4.weights')
        self.model = cv2.dnn_DetectionModel(self.net)
        self.centroid_list = []
        self.classes = None
        self.dict_color = {}
        self.classese_position = {}
        self.read_class_file()

    # Check whether points inside drew polygon or not 
    def isInsidePolygon(self, points):
        polygon = Polygon(points)
        for centroid in self.centroid_list:
            centroid = Point(centroid)
            if (polygon.contains(centroid)): return True
        return False
    
    # Check wheter detected objects inside other objects or not
    def isInsideObjects(self, objects):
        for target_obs in objects:
            if target_obs not in self.classese_position.keys(): continue
            for tbox in self.classese_position[target_obs]:
                for centroid in self.centroid_list:
                    x, y, w, h = tbox
                    if centroid[0]>=x and centroid[0]<=x+w and centroid[1]>=y and centroid[1]<=y+h:
                        return True
        return False 

    # Spride out all possible classname from Yolov4 pretrain
    def read_class_file(self):
        with open(self.classes_file, 'r') as f:
            self.classes = f.read().splitlines()
        for value in self.classes:
            self.dict_color[value]=(rd.randint(0,255), rd.randint(0,255), rd.randint(0,255))

    # Set alert when condition is satisfied
    def alert(self, img):
        cv2.putText(img, "ALARM!!!!", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        return img
    
    # Calculate the centeroid of each object
    def getCentroid(self, frame, classname, x, y, w_plus, h_plus, color=(0,255,0)):
        if classname not in self.detect_class: return frame
        centroid = ((x + w_plus) // 2, (y + h_plus) // 2)
        cv2.circle(frame, centroid, 5, color, -1)
        self.centroid_list.append(centroid)
        return frame
    
    # draw shape for each detected object
    def draw_prediction(self, frame, box, classId, score):
        if score<self.conf_threshold: return frame
        x, y, w, h = box
        cv2.rectangle(frame, (x, y), ((x + w), (y + h)), color=self.dict_color[self.classes[classId]], thickness=2)
        text = '%s: %.2f' % (self.classes[classId], score)
        cv2.putText(frame, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, color=self.dict_color[self.classes[classId]], thickness=2)
        frame = self.getCentroid(frame, self.classes[classId], x, y, (x+w), (y+h), color=self.dict_color[self.classes[classId]])
        return frame
    
    # Return result after checking
    def detect_intrusion_customPolygon(self, frame, points):
        if self.isInsidePolygon(points): return self.alert(cv2.resize(frame, (int(frame.shape[1]*self.zoom), int(frame.shape[0]*self.zoom))))
        else: return cv2.resize(frame, (int(frame.shape[1]*self.zoom), int(frame.shape[0]*self.zoom)))
    
    # Return result after checking
    def detect_instrusion_objects(self, frame, objects):
        if self.isInsideObjects(objects): return self.alert(cv2.resize(frame, (int(frame.shape[1]*self.zoom), int(frame.shape[0]*self.zoom))))
        else: return cv2.resize(frame, (int(frame.shape[1]*self.zoom), int(frame.shape[0]*self.zoom)))
    
    # detect out all the possible ojects in frame
    def detect_object(self, frame):
        self.classese_position = {}
        self.model.setInputParams(scale=self.scale, size=self.img_size, swapRB=True)
        classIds, scores, boxes = self.model.detect(frame, confThreshold=self.conf_threshold, nmsThreshold=self.nms_threshold)
        for (classId, score, box) in zip(classIds, scores, boxes):
            if self.only_detect_object and 'all' not in self.detect_class and self.classes[classId] not in self.detect_class: continue
            frame = self.draw_prediction(frame, box, classId, score)
            try: self.classese_position[self.classesId].append(box)
            except: self.classese_position[self.classes[classId]]=[box]
        return frame

After reparing main class for model detection, next is several principal functions

In [53]:
# Constructor first model
model = YoloDetect()

# Check whetehr user use mouse for drawing polygon
def handle_left_click(event, x, y, flags, points):
    if event == cv2.EVENT_LBUTTONDOWN:
        points.append([x, y])

# Connect all points to a polygon
def draw_polygon (frame, points):
    for point in points:
        cv2.circle(frame, (point[0], point[1]), 5, (0,0,255), -1)
    cv2.polylines(frame, [np.int32(points)], False, (255,0, 0), thickness=2)
    return frame

Set up camera (or input) for model detection

In [55]:
def OpenStreamCamera(points, targets_class=['person'], objects_class=['polygon'], path='stream', fps=30):
    model = YoloDetect(detect_class=targets_class, zoom=1.5)
    if path == 'stream': video = VideoStream(src=0).start()
    else: video = cv2.VideoCapture(path)
    detect = False 
    while True:
        frame = video.read()
        # Flip mirror camera
        frame = cv2.flip(frame, 1)
        key = cv2.waitKey(fps)
        draw_polygon(frame, points)
        if detect:
            frame = model.detect_object(frame)
            if objects_class==['polygon']:
                frame = model.detect_intrusion_customPolygon(frame, points)
            else: frame = model.detect_instrusion_objects(frame, objects_class)
        cv2.imshow('Frame', frame)
        cv2.setMouseCallback('Frame', handle_left_click, points)
        # Press S for breaking procedure
        if key==ord('s'): break
        # Press Q for starting procedure
        elif key == ord('q'):
            if len(points)!=0: points.append(points[0])
            detect = True
    video.stop()
    cv2.destroyAllWindows() 

- `points`: the points in current polygon
- `targets_class`: the class that we would focus on
- `objects_class`: the class that we would define approaching with `targets_class` or not

In [56]:
OpenStreamCamera([], targets_class=['cell phone'], objects_class=['person'])

Exception in thread WebcamVideoStream:
Traceback (most recent call last):
  File "c:\APP\Python\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "c:\APP\Python\lib\threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "c:\APP\Python\lib\site-packages\imutils\video\webcamvideostream.py", line 34, in update
    (self.grabbed, self.frame) = self.stream.read()
cv2.error: bad allocation
Exception in thread WebcamVideoStream:
Traceback (most recent call last):
  File "c:\APP\Python\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "c:\APP\Python\lib\threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "c:\APP\Python\lib\site-packages\imutils\video\webcamvideostream.py", line 34, in update
    (self.grabbed, self.frame) = self.stream.read()
cv2.error: bad allocation
Exception in thread WebcamVideoStream:
Traceback (most recent call last):
  File "c:\APP\Python\lib\threading.py", line 10